In [50]:
from pycalphad import Database
from pycalphad.io import tdb
from pycalphad import Database, calculate, equilibrium, variables as v

In [52]:
db_osi = Database('databases/nims_alo.tdb')
phases = []
for i in db_osi.phases:
    phases.append(i)
phases

['GAS', 'IO_LIQUID', 'BCC_A2', 'FCC_A1', 'HCP_A3', 'M2O3']

In [57]:
wksp = Workspace(
    db_osi,
    ['O', 'AL'],
    phases,
    {v.X('AL'):(0.01,0.99,0.1), v.T:1000, v.P:101325, v.N:1}
)

In [59]:
wksp.get(v.T), wksp.get(phases)

TypeError: ['GAS', 'IO_LIQUID', 'BCC_A2', 'FCC_A1', 'HCP_A3', 'M2O3'] is not a ComputableProperty